In [2]:
import pandas as pd
import sqlite3

In [1]:
DB_NAME = 'static/data/users.db'

In [3]:
conn = sqlite3.connect(DB_NAME)

In [5]:
cmd = """SELECT * from {}""".format('sid2')
df = pd.read_sql(cmd, conn)
df.head()

,Title,Date
0,New Girl: Season 2: Cooler,2/15/23
1,New Girl: Season 2: Pepperwood,2/15/23
2,New Girl: Season 2: A Father's Love,2/15/23
3,Better Call Saul: Season 5: Wexler v. Goodman,2/14/23
4,New Girl: Season 2: Cabin,2/14/23


In [8]:
def clean_watch_history(df):
    '''
    Function that cleans a given users watch history data
    Input: dataframe
    Output: (cleaned) dataframe
    '''
    df = df.rename(columns = {"Title": "History"})
    df['Date'] = pd.to_datetime(df['Date'])
    df['Day']= df['Date'].dt.day
    df['Month']= df['Date'].dt.month
    df['Year']= df['Date'].dt.year
    df['Day_of_week'] = df['Date'].dt.dayofweek

    df['Title'] = df['History'].str.rsplit(': ', 2).str[0]
    df['Season'] = df['History'].str.rsplit(': ', 2).str[1]
    df['Episode'] = df['History'].str.rsplit(': ', 2).str[2]

    df['Type'] = df['Episode'].apply(lambda x : 'Movie' if (pd.isna(x)==True) else 'TV')

    tv = df[df['Type']!='Movie']
    tv['Season'] = tv['Season'].str.split().str[1]

    movies = df[df['Type']=='Movie']
    movies['Title'] = movies['History']
    movies['Season'] = None

    df = pd.concat([movies, tv], ignore_index = True)
    return df

In [9]:
df = clean_watch_history(df)

/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_16141/3536717928.py:14: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df['Title'] = df['History'].str.rsplit(': ', 2).str[0]
/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_16141/3536717928.py:15: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df['Season'] = df['History'].str.rsplit(': ', 2).str[1]
/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_16141/3536717928.py:16: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df['Episode'] = df['History'].str.rsplit(': ', 2).str[2]
/var/folders/nx/02f7zqm17f5_3khk_h9y42s00000gn/T/ipykernel_16141/3536717928.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [10]:
df

,History,Date,Day,Month,Year,Day_of_week,Title,Season,Episode,Type
0,The Perks of Being a Wallflower,2023-01-05,5,1,2023,3,The Perks of Being a Wallflower,None,NaN,Movie
1,The Dictator,2023-01-05,5,1,2023,3,The Dictator,None,NaN,Movie
2,Haider,2023-01-05,5,1,2023,3,Haider,None,NaN,Movie
3,Collateral,2023-01-05,5,1,2023,3,Collateral,None,NaN,Movie
4,Meet the Parents,2023-01-05,5,1,2023,3,Meet the Parents,None,NaN,Movie
...,...,...,...,...,...,...,...,...,...,...
1828,Sherlock: Series 3: His Last Vow,2018-09-03,3,9,2018,0,Sherlock,3,His Last Vow,TV
1829,Sherlock: Series 3: The Sign of Three,2018-09-03,3,9,2018,0,Sherlock,3,The Sign of Three,TV
1830,Sherlock: Series 3: The Empty Hearse,2018-09-02,2,9,2018,6,Sherlock,3,The Empty Hearse,TV
1831,Sherlock: Series 2: The Reichenbach Fall,2018-09-01,1,9,2018,5,Sherlock,2,The Reichenbach Fall,TV


In [13]:
def netflix_merge(df):
    '''
    Function that merges given watch history with netflix dataset,
    and returns merged dataset
    '''
    titles = pd.read_csv('static/data/titles.csv')
    merged = df.merge(titles, left_on = 'Title', right_on = 'title', how = 'inner')
    cols_to_drop = ['type', 'production_countries', 'imdb_id', 'age_certification', 
                    'id', 'title', 'seasons', 'tmdb_popularity']
    merged = merged.drop(cols_to_drop, axis = 1)
    return merged

In [14]:
df = netflix_merge(df)

In [15]:
df

,History,Date,Day,Month,Year,Day_of_week,Title,Season,Episode,Type,description,release_year,runtime,genres,imdb_score,imdb_votes,tmdb_score
0,Haider,2023-01-05,5,1,2023,3,Haider,None,NaN,Movie,A young man returns to Kashmir after his fathe...,2014,150,"['action', 'drama', 'thriller', 'crime']",8.1,54709.0,7.100
1,Collateral,2023-01-05,5,1,2023,3,Collateral,None,NaN,Movie,When a pizza delivery driver is shot dead in s...,2018,57,"['thriller', 'drama', 'crime']",6.7,17774.0,6.400
2,Bill Burr: Live at Red Rocks,2023-01-04,4,1,2023,2,Bill Burr: Live at Red Rocks,None,NaN,Movie,Comedian Bill Burr sounds off on cancel cultur...,2022,82,"['comedy', 'documentation']",7.8,1281.0,6.900
3,The Gray Man,2022-12-28,28,12,2022,2,The Gray Man,None,NaN,Movie,When a shadowy CIA agent uncovers damning agen...,2022,128,"['thriller', 'action']",6.6,46071.0,6.851
4,Delhi Belly,2022-12-26,26,12,2022,0,Delhi Belly,None,NaN,Movie,"Three unsuspecting, average guys find themselv...",2011,102,"['comedy', 'action', 'crime', 'thriller']",7.5,29927.0,7.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026,Stranger Things: Chapter Six: The Monster,2018-09-07,7,9,2018,4,Stranger Things,Six,The Monster,TV,"When a young boy vanishes, a small town uncove...",2016,61,"['scifi', 'thriller', 'drama', 'fantasy', 'hor...",8.7,1101055.0,8.635
1027,Stranger Things: Chapter Five: The Flea and th...,2018-09-07,7,9,2018,4,Stranger Things,Five,The Flea and the Acrobat,TV,"When a young boy vanishes, a small town uncove...",2016,61,"['scifi', 'thriller', 'drama', 'fantasy', 'hor...",8.7,1101055.0,8.635
1028,Stranger Things: Chapter Four: The Body,2018-09-07,7,9,2018,4,Stranger Things,Four,The Body,TV,"When a young boy vanishes, a small town uncove...",2016,61,"['scifi', 'thriller', 'drama', 'fantasy', 'hor...",8.7,1101055.0,8.635
1029,"Stranger Things: Chapter Three: Holly, Jolly",2018-09-07,7,9,2018,4,Stranger Things,Three,"Holly, Jolly",TV,"When a young boy vanishes, a small town uncove...",2016,61,"['scifi', 'thriller', 'drama', 'fantasy', 'hor...",8.7,1101055.0,8.635


In [11]:
data.to_sql(username, conn, index = False, if_exists='replace')

277

In [14]:
cmd = f"""SELECT * FROM {username}"""
pd.read_sql(cmd, conn)

,Title,Date
0,Better Call Saul: Season 5: Something Unforgiv...,15/02/23
1,Better Call Saul: Season 5: Bad Choice Road,15/02/23
2,Better Call Saul: Season 5: Bagman,15/02/23
3,Better Call Saul: Season 5: JMM,15/02/23
4,Better Call Saul: Season 5: Dedicado a Max,14/02/23
...,...,...
272,New Girl: Season 2: Neighbors,21/11/21
273,New Girl: Season 2: Fluffer,21/11/21
274,Squid Game: Season 1: One Lucky Day,21/11/21
275,Squid Game: Season 1: VIPS,18/11/21


In [13]:
results

[('Better Call Saul: Season 5: Something Unforgivable', '15/02/23'),
 ('Better Call Saul: Season 5: Bad Choice Road', '15/02/23'),
 ('Better Call Saul: Season 5: Bagman', '15/02/23'),
 ('Better Call Saul: Season 5: JMM', '15/02/23'),
 ('Better Call Saul: Season 5: Dedicado a Max', '14/02/23'),
 ('Better Call Saul: Season 5: Namaste', '13/02/23'),
 ('Better Call Saul: Season 5: The Guy for This', '13/02/23'),
 ('Better Call Saul: Season 5: 50% Off', '12/02/23'),
 ('Better Call Saul: Season 5: Magic Man', '11/02/23'),
 ('Better Call Saul: Season 4: Winner', '10/02/23'),
 ('Better Call Saul: Season 4: Wiedersehen', '10/02/23'),
 ('Better Call Saul: Season 4: Coushatta', '10/02/23'),
 ('Better Call Saul: Season 4: Something Stupid', '09/02/23'),
 ('Better Call Saul: Season 4: Piñata', '09/02/23'),
 ('Better Call Saul: Season 4: Quite a Ride', '08/02/23'),
 ('Better Call Saul: Season 4: Talk', '08/02/23'),
 ('Better Call Saul: Season 4: Something Beautiful', '07/02/23'),
 ('Better Call Saul